## Predicting Final Round Team Compositions in Rainbow 6: Siege

#### Description of the dataset
The data used is an official datadump from Ubisoft, the developers of the game. This data was released after the fifth competitive season of gameplay (In keeping with the games' story as a counter-terrorism first-person shooter, the seasons have operation names. This particular one was named "Operation Velvet Shell"). For the project, the full datadump, featuring a detailed round-by-round breakdown of matches, was used. Each round is detailed from the perspective of each involved player, and includes details on the operator used, the exact loadout of the operator in that round, general measures of the player's skill, and specific match performance statistics, such as the number of kills, and whether they were killed in the round.

First, a subset of approximately 200,000 entries from the data will be obtained. This subset will be sampled in such a way as to preserve round/match groupings, e.g. by date. Then, the data will be aggregated and reshaped in order to combine all per-player-per-round entries into a single entry per match, detailing the choices and statistics of all involved players, as well a relevant match data, e.g. map name and game mode. This aggregated form will be the final data used for training and predictions.

#### Description of the project
Using the aggregated subset described above, a Generative Adversarial Network (GAN) will be trained to predict the team compositions of the Blue and Orange teams in the final round of a match. Since there are a large number of factors affecting a player's choice of operator in the game, such as personal preference, player skill with specific operators, the choices of other team members, and counterpicks to opponent's favourites, GANs are perfect for this role.

#### Description of GANs
TODO, also cite one of the neat articles on the subject to impress Dr. Miller more when the network inevitably fails to work properly.

In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("downsampled_datadump.csv")

In [3]:
# Check size of downsampled dataset. ~217,000 is adequately reduced.
data.shape

(217269, 31)

In [4]:
data.iloc[:5,:18]

,dateid,platform,gamemode,mapname,matchid,roundnumber,objectivelocation,winrole,endroundreason,roundduration,clearancelevel,skillrank,role,team,haswon,operator,nbkills,isdead
0,20170212,PC,PvP – HOSTAGE,CLUB HOUSE,1522380841,1,STRIP CLUB,Defender,AttackersKilledHostage,124,64,Gold,Defender,1,1,SWAT-CASTLE,0,0
1,20170212,PC,PvP – HOSTAGE,CLUB HOUSE,1522380841,4,CHURCH,Defender,AttackersEliminated,217,81,Gold,Defender,0,1,GSG9-JAGER,0,1
2,20170212,PC,PvP – HOSTAGE,CLUB HOUSE,1522380841,3,CHURCH,Defender,AttackersEliminated,160,150,Gold,Defender,1,1,JTF2-FROST,0,0
3,20170212,PC,PvP – HOSTAGE,CLUB HOUSE,1522380841,4,CHURCH,Defender,AttackersEliminated,217,94,Gold,Defender,0,1,BOPE-CAVEIRA,3,0
4,20170212,PC,PvP – HOSTAGE,CLUB HOUSE,1522380841,6,BEDROOM,Attacker,DefendersEliminated,143,81,Gold,Defender,0,0,GSG9-JAGER,0,1


#### Restructuring the Data

Given the limitations of the multi-level index in pandas, the optimal way to restructure the data into per-match rows is to construct a new dataframe. Additionally, as repeated queries are quite slow, an intermediate storage form of the data will be constructed, as a nested dictionary containing nested lists of the form: `{<matchid>: {<roundnumber>: [[row1], [row2], ...]}}` This intermediate representation will then be used to construct the new dataframe, as it will be both more computationally efficient and enable easier filtering of non-structurally compliant rounds.

The new structure requires that games have exactly six rounds played, as well as a full complement of five players, in order to standardise the length of input data to the GAN. Additionally, a number of details about individual player loadouts will be dropped, as the exact set of mods on each weapon is almost purely dictacted by the meta. Of more interest is the general weapon types selected by each op, even though this is still largely meta-dependent.

In [5]:
# The intermediate step of aggregating by round prior to aggregating into matches is no longer being used,
# but setting up the data dictionary for it was very helpful in figuring out how to structure the new
# dataframe of aggregated data.

# per_round = {"gamemode": [], "mapname": [], "matchid": [], "roundnumber": [], "objectivelocation": [],
#              "winrole": [], "endroundreason": [], "roundduration": [], "bluerole": [], "orangerole": [],
#              "teamwin": [], "blue1skill": [], "blue1level": [], "blue1kills": [], "blue1dead": [],
#              "blue1op": [], "blue1primary": [], "blue1secondary": [], "blue2skill": [],
#              "blue2level": [], "blue2kills": [], "blue2dead": [], "blue2op": [], "blue2primary": [],
#              "blue2secondary": [], "blue3skill": [], "blue3level": [], "blue3kills": [],
#              "blue3dead": [], "blue3op": [], "blue3primary": [], "blue3secondary": [],
#              "blue4skill": [], "blue4level": [], "blue4kills": [], "blue4dead": [], "blue4op": [],
#              "blue4primary": [], "blue4secondary": [], "blue5skill": [], "blue5level": [],
#              "blue5kills": [], "blue5dead": [], "blue5op": [], "blue5primary": [], "blue5secondary": [],
#              "orange1skill": [], "orange1level": [], "orange1kills": [], "orange1dead": [],
#              "orange1op": [], "orange1primary": [], "orange1secondary": [], "orange2skill": [],
#              "orange2level": [], "orange2kills": [], "orange2dead": [], "orange2op": [],
#              "orange2primary": [], "orange2secondary": [], "orange3skill": [], "orange3level": [],
#              "orange3kills": [], "orange3dead": [], "orange3op": [], "orange3primary": [],
#              "orange3secondary": [], "orange4skill": [], "orange4level": [], "orange4kills": [],
#              "orange4dead": [], "orange4op": [], "orange4primary": [], "orange4secondary": [],
#              "orange5skill": [], "orange5level": [], "orange5kills": [], "orange5dead": [],
#              "orange5op": [], "orange5primary": [], "orange5secondary": [],
#             }

# Arbitrary decision: Blue is 0, Orange is 1.

In [6]:
# ~Elegant~ programmatic data dictionary creation
per_match = {"gamemode": [], "mapname": [], "matchid": []}
for r_num in range(1, 7):
    for detail in ["objective", "endreason", "duration", "bluerole", "orangerole", "winner", "winrole"]:
        per_match["".join(["round", str(r_num), detail])] = []
    for team in ["blue", "orange"]:
        for player in range(1,6):
            for field in ["level", "skill", "kills", "dead", "op", "primary", "secondary"]:
                per_match["".join(["round", str(r_num), team, str(player), field])] = []

In [7]:
tmp_storage = {}
data.columns

Index(['dateid', 'platform', 'gamemode', 'mapname', 'matchid', 'roundnumber',
       'objectivelocation', 'winrole', 'endroundreason', 'roundduration',
       'clearancelevel', 'skillrank', 'role', 'team', 'haswon', 'operator',
       'nbkills', 'isdead', 'primaryweapon', 'primaryweapontype',
       'primarysight', 'primarygrip', 'primaryunderbarrel', 'primarybarrel',
       'secondaryweapon', 'secondaryweapontype', 'secondarysight',
       'secondarygrip', 'secondaryunderbarrel', 'secondarybarrel',
       'secondarygadget'],
      dtype='object')

In [8]:
%%time
for row in data.values:
    matchid = row[4]
    r_num = row[5]
    if matchid not in tmp_storage:
        tmp_storage[matchid] = {}
    if r_num not in tmp_storage[matchid]:
        tmp_storage[matchid][r_num] = [row]
    else:
        tmp_storage[matchid][r_num].append(row)

CPU times: user 444 ms, sys: 88 ms, total: 532 ms
Wall time: 530 ms


In [9]:
tmp_storage[1522380841]

{1: [array([20170212, 'PC', 'PvP – HOSTAGE', 'CLUB HOUSE', 1522380841, 1,
         'STRIP CLUB', 'Defender', 'AttackersKilledHostage', 124, 64,
         'Gold', 'Defender', 1, 1, 'SWAT-CASTLE', 0, 0, 'UMP45',
         'Submachine Guns', 'RedDot', 'Vertical', 'None', 'Compensator',
         '5.7 USG', 'Pistols', 'None', 'None', 'None', 'None',
         'IMPACT GRENADE'], dtype=object),
  array([20170212, 'PC', 'PvP – HOSTAGE', 'CLUB HOUSE', 1522380841, 1,
         'STRIP CLUB', 'Defender', 'AttackersKilledHostage', 124, 120,
         'Gold', 'Attacker', 0, 0, 'GIGN-TWITCH', 0, 1, 'F2',
         'Assault Rifles', 'Acog', 'Vertical', 'None', 'Compensator', 'P9',
         'Pistols', 'None', 'None', 'None', 'MuzzleBrake', 'BREACH CHARGE'],
        dtype=object),
  array([20170212, 'PC', 'PvP – HOSTAGE', 'CLUB HOUSE', 1522380841, 1,
         'STRIP CLUB', 'Defender', 'AttackersKilledHostage', 124, 150,
         'Gold', 'Defender', 1, 1, 'JTF2-FROST', 1, 0, '9mm C1',
         'Submachine Guns